In [141]:
import talib

In [142]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [143]:
data  = pd.read_csv('BTCUSD.csv')
data.head()

,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,BTC,2020-03-24,6455.454688,5829.352511,6620.858253,5694.198299
1,BTC,2020-03-25,6784.318011,6455.450650,6863.602196,6406.037439
2,BTC,2020-03-26,6706.985089,6784.325204,6981.720386,6488.111885
3,BTC,2020-03-27,6721.495392,6697.948320,6796.053701,6537.856462
4,BTC,2020-03-28,6682.780049,6718.084872,6865.541424,6597.307584


In [144]:
data.shape

(549, 6)

In [ ]:
from sklearn.model_selection import train_test_split

In [145]:
data.columns

Index(['Currency', 'Date', 'Closing Price (USD)', '24h Open (USD)',
       '24h High (USD)', '24h Low (USD)'],
      dtype='object')

In [146]:
data['Closing Price (USD)'].values.shape

(549,)

In [147]:
scalar = MinMaxScaler(feature_range=(0,1))
scaled_data = scalar.fit_transform(data['Closing Price (USD)'].values.reshape(-1, 1))
# scaled_data = scaled_data.tolist()
# scaled_data[:5] 

In [148]:
time_intervals_to_train = 24 # 5 min intervals. 1 Hour = 60 min = 12 intervals.
prediction_interval = 12 

x_train = [] #it will be list of multiple arrays
y_train = [] #it will be list of multiple arrays

for i in range(time_intervals_to_train, len(scaled_data) -  prediction_interval):
    x_train.append(scaled_data[i - time_intervals_to_train : i, 0])
    y_train.append(scaled_data[i + prediction_interval , 0])

x_train = np.array(x_train)
y_train = np.array(y_train)

In [149]:
# x_train = np.stack([arr for arr in x_train if len(arr) == 24])
x_train.shape


(513, 24)

In [150]:
# reshape x_train to fit LSTM mode
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(513, 24, 1)

In [151]:
y_train.shape
y_train[:5]

array([0.03202945, 0.0496413 , 0.04954667, 0.05104607, 0.05294915])

In [152]:
#create Model
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape = (x_train.shape[1], 1), activation = "relu"))
model.add(Dropout(0.4))
model.add(LSTM(64, return_sequences = True,  activation = "relu"))
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences = True,  activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "sigmoid"))

In [153]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 24, 128)           66560     
                                                                 
 dropout_6 (Dropout)         (None, 24, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 24, 64)            49408     
                                                                 
 dropout_7 (Dropout)         (None, 24, 64)            0         
                                                                 
 lstm_9 (LSTM)               (None, 24, 32)            12416     
                                                                 
 dropout_8 (Dropout)         (None, 24, 32)            0         
                                                                 
 dense_2 (Dense)             (None, 24, 1)            

In [154]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [155]:
x_train.shape

(513, 24, 1)

In [159]:
y_train.shape

(513,)

In [157]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
9/9 [==============================] - 5s 65ms/step - loss: 0.1011 - accuracy: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 1s 66ms/step - loss: 0.0979 - accuracy: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 1s 64ms/step - loss: 0.0963 - accuracy: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 1s 63ms/step - loss: 0.0961 - accuracy: 0.0000e+00
Epoch 5/10
9/9 [==============================] - 1s 66ms/step - loss: 0.0955 - accuracy: 0.0000e+00
Epoch 6/10
9/9 [==============================] - 1s 69ms/step - loss: 0.0954 - accuracy: 0.0000e+00
Epoch 7/10
9/9 [==============================] - 1s 68ms/step - loss: 0.0953 - accuracy: 0.0000e+00
Epoch 8/10
9/9 [==============================] - 1s 64ms/step - loss: 0.0953 - accuracy: 0.0000e+00
Epoch 9/10
9/9 [==============================] - 1s 64ms/step - loss: 0.0957 - accuracy: 0.0000e+00
Epoch 10/10
9/9 [==============================] - 1s 67ms/step - loss: 0.0956 - accuracy: 

In [158]:
data.columns

Index(['Currency', 'Date', 'Closing Price (USD)', '24h Open (USD)',
       '24h High (USD)', '24h Low (USD)'],
      dtype='object')